<a href="https://colab.research.google.com/github/Android998/TensorFlow2.3/blob/master/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Paso 1: Instalacion de dependencias y configuración del entorno CPU**

In [3]:
!pip install tensorflow-gpu==2.3

     |████████████████████████████████| 320.4MB 50kB/s 


In [4]:
!pip install tqdm

#### **Descarga el dataset de Perros vs Gatos**

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-09-12 12:55:37--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 74.125.128.128, 74.125.143.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  94.2MB/s    in 0.7s    

2020-09-12 12:55:39 (94.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



### **Paso 2: Pre procesado de datos**

#### **Importar las dependencias del proyecto**

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

#### **Descomprimir el fichero de Perros vs Gatos**

In [5]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [6]:
zip_object = zipfile.ZipFile(file = dataset_path, mode = "r")

In [7]:
zip_object.extractall("./")

In [8]:
zip_object.close()

#### **Configurar las rutas del dataset**

In [9]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [10]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

### **Paso 3: Construir el modelo**

#### **Cargar un modelo preentrenado(MobileNetV2)**

In [11]:
IMG_SHAPE = (128, 128, 3)

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

#### **Congelar el modelo base**

In [14]:
base_model.trainable = False

#### **Definir la cabecera personalizada para nuestra red neuronal**

In [15]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [18]:
prediction_layer = tf.keras.layers.Dense(units=1, activation = "sigmoid")(global_average_layer)

#### **Definir el modelo**

In [19]:
model = tf.keras.models.Model(inputs=base_model.input, outputs = prediction_layer)

In [20]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

#### **Compilar el modelo**

In [21]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"] )

#### **Crear generadores de datos**
Redimensionar imagenes:

*Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imagenes.*

Por ejemplo: MobileNet soporta: (96,96), (128,128), (160, 160), (192, 192), (224,224)

In [22]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_valid = ImageDataGenerator(rescale=1/255.0)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128, 128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generation = data_gen_valid.flow_from_directory(validation_dir, target_size=(128, 128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


#### **Entrenar el modelo**

In [25]:
model.fit(train_generator, epochs=20, validation_data=valid_generation)

Epoch 1/20
16/16 [==============================] - 8s 491ms/step - loss: 0.7519 - accuracy: 0.5415 - val_loss: 0.6921 - val_accuracy: 0.6070
Epoch 2/20
16/16 [==============================] - 7s 444ms/step - loss: 0.6313 - accuracy: 0.6510 - val_loss: 0.5975 - val_accuracy: 0.7020
Epoch 3/20
16/16 [==============================] - 7s 437ms/step - loss: 0.5482 - accuracy: 0.7270 - val_loss: 0.5206 - val_accuracy: 0.7630
Epoch 4/20
16/16 [==============================] - 7s 438ms/step - loss: 0.4782 - accuracy: 0.7855 - val_loss: 0.4548 - val_accuracy: 0.8140
Epoch 5/20
16/16 [==============================] - 7s 435ms/step - loss: 0.4199 - accuracy: 0.8405 - val_loss: 0.4004 - val_accuracy: 0.8540
Epoch 6/20
16/16 [==============================] - 7s 441ms/step - loss: 0.3705 - accuracy: 0.8725 - val_loss: 0.3539 - val_accuracy: 0.8790
Epoch 7/20
16/16 [==============================] - 7s 433ms/step - loss: 0.3301 - accuracy: 0.8940 - val_loss: 0.3145 - val_accuracy: 0.9090
Epoch 

#### **Evaluar el modelo de aprendizaje por transferencia**

In [26]:
valid_loss, valid_accuracy = model.evaluate(valid_generation)

8/8 [==============================] - 2s 266ms/step - loss: 0.1308 - accuracy: 0.9600


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9599999785423279


### **Puesta a punto de parámetros**


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

#### **Descongelar unas cuantas capas superiores del modelo**

In [28]:
base_model.trainable = True

In [29]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [39]:
fine_tune_at = 130

In [40]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### **Compilar el modelo para la puesta a punto**

In [41]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

#### **Puesta a punto del modelo**

In [42]:
model.fit(train_generator, epochs=10, validation_data=valid_generation)

Epoch 1/10
16/16 [==============================] - 8s 498ms/step - loss: 2.8222e-04 - accuracy: 1.0000 - val_loss: 0.2422 - val_accuracy: 0.9600
Epoch 2/10
16/16 [==============================] - 7s 450ms/step - loss: 0.0011 - accuracy: 0.9995 - val_loss: 0.4611 - val_accuracy: 0.9330
Epoch 3/10
16/16 [==============================] - 7s 437ms/step - loss: 1.1088e-04 - accuracy: 1.0000 - val_loss: 0.4316 - val_accuracy: 0.9380
Epoch 4/10
16/16 [==============================] - 7s 438ms/step - loss: 2.1377e-04 - accuracy: 1.0000 - val_loss: 0.4704 - val_accuracy: 0.9310
Epoch 5/10
16/16 [==============================] - 7s 435ms/step - loss: 9.0790e-05 - accuracy: 1.0000 - val_loss: 0.4662 - val_accuracy: 0.9380
Epoch 6/10
16/16 [==============================] - 7s 432ms/step - loss: 3.6639e-05 - accuracy: 1.0000 - val_loss: 0.3872 - val_accuracy: 0.9480
Epoch 7/10
16/16 [==============================] - 7s 437ms/step - loss: 1.8962e-05 - accuracy: 1.0000 - val_loss: 0.3913 - val

#### **Evaluar el modelo**

In [36]:
valid_loss, valid_accuracy = model.evaluate(valid_generation)

8/8 [==============================] - 2s 264ms/step - loss: 0.2334 - accuracy: 0.9650


In [37]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9649999737739563
